# Map Visualisation of the Dataset

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

from shipsnet.data import load_data_from_json
from shipsnet.viz import array_to_rgb_image

%load_ext jupyter_black

In [ ]:
data_dict = load_data_from_json()

data_df = pd.DataFrame(data_dict)
data_df.info()
data_df.head()

In [ ]:
lon, lat = np.array(data_dict["locations"]).T

df = data_df.drop(columns=["locations", "data"])
df["lon"] = lon
df["lat"] = lat

df["labels"] = df.labels.apply(lambda label: "ship" if label else "no ship")

df.head()

# Quick visualisation using Plotly

In [ ]:
import plotly.express as px

In [ ]:
px.scatter_geo(
    df,
    lat="lat",
    lon="lon",
    hover_name="scene_ids",
    scope="usa",
    title="image locations",
)

Looks like there's two clusters at San Francisco and Los Angeles. Let's split the dataset into these clusters.

In [ ]:
df_sf = df[df.lat > 35]  # san francisco
df_la = df[df.lat < 35]  # los angeles

In [ ]:
fig = px.scatter_geo(
    df_sf[::-1],  # so ship images are on top of no-ship images
    lat="lat",
    lon="lon",
    hover_name="scene_ids",
    color="labels",
    color_discrete_map={"ship": "red", "no ship": "grey"},
    fitbounds="locations",
    scope="north america",
    title="Image Locations: San Francisco Bay",
)
fig.update_geos(
    resolution=50,
    showcoastlines=True,
    coastlinecolor="RebeccaPurple",
    showland=True,
    landcolor="LightGreen",
    showocean=True,
    oceancolor="LightBlue",
)
fig.update_layout(height=300, margin={"r": 0, "t": 50, "l": 0, "b": 0})

In [ ]:
fig = px.scatter_geo(
    df_la[::-1],
    lat="lat",
    lon="lon",
    hover_name="scene_ids",
    color="labels",
    color_discrete_map={"ship": "red", "no ship": "grey"},
    fitbounds="locations",
    scope="north america",
    title="Image Locations: San Pedro Bay",
)
fig.update_geos(
    resolution=50,
    showcoastlines=True,
    coastlinecolor="RebeccaPurple",
    showland=True,
    landcolor="LightGreen",
    showocean=True,
    oceancolor="LightBlue",
)
fig.update_layout(height=300, margin={"r": 0, "t": 50, "l": 0, "b": 0})

# Interactive Leaflet Map

In [ ]:
from ipywidgets import HTML, Image
from ipyleaflet import (
    Map,
    basemaps,
    GeoData,
    WidgetControl,
)
import geopandas as gpd

images_dir = Path("../data/shipsnet/shipsnet")

In [ ]:
df_ = df_la  # choose which dataset to plot

leafmap = Map(
    center=(df_.lat.mean(), df_.lon.mean()),
    zoom=10,
)

gdf = gpd.GeoDataFrame(df_, geometry=gpd.points_from_xy(df_.lon, df_.lat))

data = GeoData(
    geo_dataframe=gdf,
    point_style={
        "radius": 5,  # point size
        "weight": 2,  # edge width
        "color": "green",  # edge colour
        "opacity": 1,  # edge opacity
        "fillColor": "green",
        "fillOpacity": 0.2,
    },
    hover_style={"color": "red"},
)
leafmap.add_layer(data)


# HTML info box
html = HTML("Click any point to display image info")
html_control = WidgetControl(widget=html, position="bottomright")
leafmap.add_control(html_control)

image = Image(value=b"", format="png", width=80, height=80)
im_control = WidgetControl(widget=image, position="bottomleft")
leafmap.add_control(im_control)


def on_click(feature, **kwargs):
    label = feature["properties"]["labels"]
    scene_id = feature["properties"]["scene_ids"]
    lon = feature["properties"]["lon"]
    lat = feature["properties"]["lat"]
    html.value = f"""
<pre>
<b>Label</b>     : {label}
<b>Scene ID</b>  : {scene_id}
<b>Latitude</b>  : {lat}
<b>Longitude</b> : {lon}
</pre>
"""
    lab = 1 if label == "ship" else 0
    filename = f"{lab}__{scene_id}__{lon}_{lat}.png"
    with images_dir.joinpath(filename).open("rb") as file:
        image.value = file.read()


data.on_click(on_click)

leafmap